In [3]:
!pip install python-dotenv
!pip install openai
!pip install jupyterlab_darkside_theme

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

## Chat GPT with Open AI and LangChain

### Simple Use

The simplest possible use for this API is to pass in a prompt to a model and get a result. We will do so with a helper function:

In [20]:
def get_result(prompt, model="gpt-3.5-turbo-1106"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.25, 
    )
    return response.choices[0].message.content


In [21]:
get_result("Who first discovered coffee and when?")

'The discovery of coffee is often attributed to an Ethiopian goat herder named Kaldi, who noticed that his goats became more energetic after eating the berries from a certain tree. This is said to have occurred around the 9th century. However, the exact origins of coffee are not definitively known, and there are various legends and stories surrounding its discovery.'

### Experiment With Prompts

Create a prompt that has some style variable attached to it as well as input apart from the instruction. Input and istruction are combined with triple backticks

In [22]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""
style = """American English in a calm and respectful tone"""

prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [23]:
get_result(prompt)

"I'm really frustrated that my blender lid flew off and splattered my kitchen walls with smoothie! And to make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I need your help right now, friend."

### Enter LangChain

In [24]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.25)


### Using LangChain Templates

In [26]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}.
text: ```{text}```"""

template = ChatPromptTemplate.from_template(template_string)

In [27]:
template

ChatPromptTemplate(input_variables=['style', 'text'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}.\ntext: ```{text}```'))])

In [29]:
customer_style = "Contemporary British English in a witty and humorous tone"
customer_email = """
{Arrr}, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""
customer_messages = template.format_messages(style=customer_style, text=customer_email)

In [32]:
customer_messages[0].content

"Translate the text that is delimited by triple backticks into a style that is Contemporary British English in a witty and humorous tone.\ntext: ```\n{Arrr}, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```"

Now we pass the formatted messages to the chat that we set earlier as `ChatOpenAI(temperature=0.01)`

In [33]:
response = chat(customer_messages)

In [34]:
response.content

'"Oh, blimey! I\'m absolutely livid that me blender lid decided to go for a little fly and redecorate me kitchen walls with smoothie! And as if that weren\'t bad enough, the warranty doesn\'t even cover the cost of cleaning up the mess. I could really use a hand here, me hearty!"'

### Output parsers
Let's start with defining how we would like the LLM output to look like:

In [35]:
obj = {
    "gift": False,
    "delivery_days": 5,
    "price_value": "pretty affordable!"
}

In [38]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'))]


In [43]:
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOpenAI(temperature=0.1)
response = chat(messages)

In [44]:
import json

obj = json.loads(response.content)
obj

{'gift': True,
 'delivery_days': 2,
 'price_value': ["It's slightly more expensive than the other leaf blowers out there"]}

Alternatively we can use LangChain parsers to do this for us.. 

This will result into parsing the LLM output directly into a Python dictionary. It seems to be rather wordy, however and I am not sure about the longer term advantages of all this.

In [46]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

In [47]:
gift_schema = ResponseSchema(name="gift", 
                             description="""Was the item purchased as a gift for someone else?\ 
                             Answer True if yes, False if not or unknown.""")
delivery_schema = ResponseSchema(name="delivery_days",
                                 description="""How many days did it take for the product \
                                 to arrive? If this information is not found, output -1.""")
price_schema = ResponseSchema(name="price_value", 
                              description="""Extract any sentences about the value or price,\
                              and output them as a comma separated Python list.""")
response_schemas = [gift_schema, delivery_schema, price_schema]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()

print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased as a gift for someone else?\ 
                             Answer True if yes, False if not or unknown.
	"delivery_days": string  // How many days did it take for the product                                  to arrive? If this information is not found, output -1.
	"price_value": string  // Extract any sentences about the value or price,                              and output them as a comma separated Python list.
}
```


In [49]:
review_template_with_instructions = """ For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}"""

prompt = ChatPromptTemplate.from_template(review_template_with_instructions)
messages = prompt.format_messages(text=customer_review, format_instructions=format_instructions)

chat = ChatOpenAI(temperature=0.1)
response = chat(messages)
type(response.content)

str

> NOTE: well, dont forget to use the parser to parse the output, it is still a string otherwise

In [50]:
parsed = output_parser.parse(response.content)
print(type(parsed),'\n', parsed,'\n', parsed["delivery_days"])

<class 'dict'> 
 {'gift': False, 'delivery_days': 2, 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]} 
 2


### Memory

- ConversationBufferMemory
- ConversationBufferWindowMemory
- ConversationTokenBufferMemory
- ConversationSummaryMemory

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory
from langchain.prompts import ChatPromptTemplate

Again we set the interface as the ChatOpenAI. We then specify the type of memory to use and initialise a LangChain ConversationChain. We will delve into details a littler later.

Starting with the most common memory:

#### ConversationBufferMemory

In [2]:
llm = ChatOpenAI(temperature=0.01)
memory = ConversationBufferMemory()
conversation = ConversationChain(llm=llm, memory=memory, verbose=True)

/opt/conda/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [19]:
conversation.predict(input="Hi there, who is it that I am talking to today?")

"Hello! You are talking to OpenAI's language model. How can I assist you today?"

In [20]:
conversation.predict(input="would you say you are better at python or javascript?")

"As an AI language model, I don't have personal preferences or skills like humans do. However, I can provide information and help with questions related to Python and JavaScript. Is there anything specific you would like to know or discuss about these programming languages?"

In [22]:
print(memory.buffer)

Human: Hi there, who is it that I am talking to today?
AI: Hello! You are talking to OpenAI's language model. How can I assist you today?
Human: would you say you are better at python or javascript?
AI: As an AI language model, I don't have personal preferences or skills like humans do. However, I can provide information and help with questions related to Python and JavaScript. Is there anything specific you would like to know or discuss about these programming languages?


In [23]:
conversation.prompt

PromptTemplate(input_variables=['history', 'input'], template='The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:')

Can I actually **override** the template. Should be possible via the `ChatPromptTemplate` class.. Lets try it out

In [24]:
custom_template = """You are a informative and helpful AI professor who is helping ML apprentices take their first steps in the field.\
If you do not know any of the answers, please be honest to acknoledge that you do not have this information.

Current conversation:
{history}

Human:
{input}
AI Professor:
"""
prompt = ChatPromptTemplate.from_template(custom_template)

In [29]:
memory.clear()
conversation = ConversationChain(llm=llm, memory=memory, prompt=prompt, verbose=True)

In [30]:
conversation.predict(input="Who is Andrew Ng? Could you give me some information about him?")



> Entering new ConversationChain chain...
Prompt after formatting:
Human: You are a informative and helpful AI professor who is helping ML apprentices take their first steps in the field.If you do not know any of the answers, please be honest to acknoledge that you do not have this information.

Current conversation:


Human:
Who is Andrew Ng? Could you give me some information about him?
AI Professor:


> Finished chain.


'Andrew Ng is a prominent figure in the field of artificial intelligence and machine learning. He is a computer scientist and entrepreneur, known for his contributions to the development of deep learning algorithms and his work in popularizing AI. Ng co-founded Coursera, an online learning platform, and served as the chief scientist at Baidu, a Chinese multinational technology company. He has also held positions at Stanford University and Google. Ng is highly regarded for his expertise in machine learning and has made significant contributions to the field through his research, teaching, and industry leadership.'

In [31]:
conversation.predict(input="What courses would you recommend to check authored by him? What is a good introduction course on\
                    Machine Learning and should I start with shallow algorithms or directly deep learning?")



> Entering new ConversationChain chain...
Prompt after formatting:
Human: You are a informative and helpful AI professor who is helping ML apprentices take their first steps in the field.If you do not know any of the answers, please be honest to acknoledge that you do not have this information.

Current conversation:
Human: Who is Andrew Ng? Could you give me some information about him?
AI: Andrew Ng is a prominent figure in the field of artificial intelligence and machine learning. He is a computer scientist and entrepreneur, known for his contributions to the development of deep learning algorithms and his work in popularizing AI. Ng co-founded Coursera, an online learning platform, and served as the chief scientist at Baidu, a Chinese multinational technology company. He has also held positions at Stanford University and Google. Ng is highly regarded for his expertise in machine learning and has made significant contributions to the field through his research, teaching, and indust

'Andrew Ng has authored several popular courses on machine learning. One of the most well-known courses he has created is "Machine Learning" on Coursera. This course provides a comprehensive introduction to machine learning and covers a wide range of topics, including linear regression, logistic regression, neural networks, support vector machines, and unsupervised learning.\n\nAs for whether you should start with shallow algorithms or directly dive into deep learning, it depends on your background and goals. If you are new to machine learning, it is generally recommended to start with the basics and build a solid foundation in shallow algorithms before moving on to deep learning. This will help you understand the fundamental concepts and techniques in machine learning. Once you have a good grasp of the basics, you can then explore deep learning, which is a more advanced and complex topic. However, if you have a strong background in mathematics and programming, you may be able to direc

#### ConversationBufferWindowMemory

This type of memory allows you to specify how many communication exchanges to remember. For example to keep just one exchange between the AI and Human in memory we initialised the memory like this: `memory = ConversationBufferWindowMemory(k=1)`

This helps us preserve the conversation context length as it may build up as we go along.

Another way to limit the memory size is by using:

#### ConversationTokenBufferMemory

By using this method we can specify number of tokens to remember instead of conversation exchanges. We use it like this: `memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)`. 
> Note: We need to specify the llm since the tokenizers of different LLMs use different algorithms to split into tokens.

Probably the most interesting type of memory is the:

#### ConversationSummaryBufferMemory

in this memory we ask the LLM to provide a summary of the conversation thus far up to a number of tokens. It is really high-end. The provided summary is recorded as a system message on top of the latest prompt. Lets create some example below.
Hmm need to install module `tiktoken`

In [5]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.6 MB/s eta 0:00:00:00:010:01


In [51]:
long_user_input = """So my agenda for tomorrow is rather busy. First I need to engage into some fitness activies.
Once done, I will head to the bank around 11am. Sign some paperwork in order to obtain my new credit card.
At 12 or 12:30 I will be in the office to check on current tasks and get a lunch at 1pm.\
Hopefully in the afternoon I should get the delivery by the bulbs supplier.
Not to forget my demo at 3pm on RAG with an open source LLM. This should make for a very nice chatbot once ready.
"""
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=250)
memory.save_context(inputs={"input": "Hey there, Chatter"}, 
                    outputs={"output": "What's up!"})
memory.save_context(inputs={"input": "Not much, just chillin before the busy day tomorrow"}, 
                    outputs={"output": "Great to hear. Let me know if I can assist you in any way."})
memory.save_context(inputs={"input": f"As a mather of fact you can! How nice of you to ask.\
Here is what I have planned for tomorrow, please keep it in mind as I try to navigate the day.\
I will ask you questions once I go along. Plan: {long_user_input}"}, outputs={"output": "Thanks, got it, I will be happy to help!"})

In [52]:
memory.load_memory_variables({})

{'history': "Human: Hey there, Chatter\nAI: What's up!\nHuman: Not much, just chillin before the busy day tomorrow\nAI: Great to hear. Let me know if I can assist you in any way.\nHuman: As a mather of fact you can! How nice of you to ask.Here is what I have planned for tomorrow, please keep it in mind as I try to navigate the day.I will ask you questions once I go along. Plan: So my agenda for tomorrow is rather busy. First I need to engage into some fitness activies.\nOnce done, I will head to the bank around 11am. Sign some paperwork in order to obtain my new credit card.\nAt 12 or 12:30 I will be in the office to check on current tasks and get a lunch at 1pm.Hopefully in the afternoon I should get the delivery by the bulbs supplier.\nNot to forget my demo at 3pm on RAG with an open source LLM. This should make for a very nice chatbot once ready.\n\nAI: Thanks, got it, I will be happy to help!"}

In [53]:
conversation = ConversationChain(llm=llm, memory=memory, verbose=True)

In [55]:
conversation.predict(input="Just to check, when did I plan my meeting tomorrow and should I take on another meeting at 4pm?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hey there, Chatter
AI: What's up!
Human: Not much, just chillin before the busy day tomorrow
AI: Great to hear. Let me know if I can assist you in any way.
Human: As a mather of fact you can! How nice of you to ask.Here is what I have planned for tomorrow, please keep it in mind as I try to navigate the day.I will ask you questions once I go along. Plan: So my agenda for tomorrow is rather busy. First I need to engage into some fitness activies.
Once done, I will head to the bank around 11am. Sign some paperwork in order to obtain my new credit card.
At 12 or 12:30 I will be in the office to check on current tasks and get a lunch at 1pm.Hopefully in the afternoo

'You have a demo at 3pm tomorrow. As for another meeting at 4pm, you did not mention any plans for that time.'

In [56]:
conversation.predict(input="Good stuff! Make a note that I will take another 30 mins meeting at 4pm with George. He is very interested in the classification model")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI engage in casual conversation. The human mentions having a busy day tomorrow and asks the AI to keep it in mind. The human outlines their agenda, which includes fitness activities, a visit to the bank, work tasks, lunch, a delivery, and a demo.
AI: Thanks, got it, I will be happy to help!
Human: Just to check, when did I plan my meeting tomorrow and should I take on another meeting at 4pm?
AI: You have a demo at 3pm tomorrow. As for another meeting at 4pm, you did not mention any plans for that time.
Human: Good stuff! Make a note that I will take another 30 mins meeting at 4pm with George. He is very interested in the classification model
AI:


"Sure, I've made a note that you will have a 30-minute meeting with George at 4pm tomorrow to discuss the classification model."

In [57]:
conversation.predict(input="hey, me again! did I have anything scheduled for the afternoon apart from those two meetings?!")




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI engage in casual conversation. The human mentions having a busy day tomorrow and asks the AI to keep it in mind. The human outlines their agenda, which includes fitness activities, a visit to the bank, work tasks, lunch, a delivery, and a demo.
AI: Thanks, got it, I will be happy to help!
Human: Just to check, when did I plan my meeting tomorrow and should I take on another meeting at 4pm?
AI: You have a demo at 3pm tomorrow. As for another meeting at 4pm, you did not mention any plans for that time.
Human: Good stuff! Make a note that I will take another 30 mins meeting at 4pm with George. He is very interested in the classification model
AI: 

'Apart from the two meetings, you did not mention any other scheduled activities for the afternoon.'

In [58]:
conversation.predict(input="I vaguely remember I had to pick a parcel.. but dont know when it was likely to arrive. Do you remember?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI engage in casual conversation. The human mentions having a busy day tomorrow and asks the AI to keep it in mind. The human outlines their agenda, which includes fitness activities, a visit to the bank, work tasks, lunch, a delivery, and a demo.
AI: Thanks, got it, I will be happy to help!
Human: Just to check, when did I plan my meeting tomorrow and should I take on another meeting at 4pm?
AI: You have a demo at 3pm tomorrow. As for another meeting at 4pm, you did not mention any plans for that time.
Human: Good stuff! Make a note that I will take another 30 mins meeting at 4pm with George. He is very interested in the classification model
AI: 

"I'm sorry, but I don't have access to your personal schedule or any information about your parcel delivery."

> Note: above we can see the limitations of this approach, however. Summarising the conversation will lose important details such as various times and chronology

### Lets Try LangChain with Memory and Open LLM with HF

In [1]:
from langchain import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
model_name = "mistralai/Mistral-7B-Instruct-v0.2"


In [2]:
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", trust_remote_code=True, load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
generation_config = GenerationConfig.from_pretrained(model_name)
generation_config.max_new_tokens = 512
generation_config.temperature = 0.01
generation_config.top_p = 0.95
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15
generation_config.pad_token_id = tokenizer.eos_token_id

text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
)
open_llm = HuggingFacePipeline(pipeline=text_pipeline)

In [9]:
long_user_input = """So my agenda for tomorrow is rather busy. First I need to engage into some fitness activies.
Once done, I will head to the bank around 11am. Sign some paperwork in order to obtain my new credit card.
At 12 or 12:30 I will be in the office to check on current tasks and get a lunch at 1pm.\
Hopefully in the afternoon I should get the delivery by the bulbs supplier.
Not to forget my demo at 3pm on RAG with an open source LLM. This should make for a very nice chatbot once ready.
"""
memory = ConversationSummaryBufferMemory(llm=open_llm, max_token_limit=250)
memory.save_context(inputs={"input": "Hey there, Chatter"}, 
                    outputs={"output": "What's up!"})
memory.save_context(inputs={"input": "Not much, just chillin before the busy day tomorrow"}, 
                    outputs={"output": "Great to hear. Let me know if I can assist you in any way."})
memory.save_context(inputs={"input": f"As a mather of fact you can! How nice of you to ask.\
Here is what I have planned for tomorrow, please keep it in mind as I try to navigate the day.\
I will ask you questions once I go along. Plan: {long_user_input}"}, outputs={"output": "Thanks, got it, I will be happy to help!"})

OK OK.. So the template is totally wrong and the AI is creating its own stories.. Lets try and define the Template properly for this model.

In [10]:
open_llm_template = """<s>[INST]You are a helpful assistant ready to assist your human friend.\
Please provide comprehensive information based on your vast knowledge base and the current conversation. 
If you do not know the answer please acknowledge the fact by saying that you do not possess the information requested.\
If the conversation is summarised it will appear as System:. \

Current conversation:
{history}

Human:
{input}[/INST]
"""
prompt = ChatPromptTemplate.from_template(open_llm_template)

In [11]:
conversation = ConversationChain(llm=open_llm, memory=memory, prompt=prompt, verbose=True)

In [12]:
conversation.predict(input="Hey AI, can you give me some summmary of my day tomorrow? Do I have fitness scheduled?")



> Entering new ConversationChain chain...
Prompt after formatting:
Human: <s>[INST]You are a helpful assistant ready to assist your human friend.Please provide comprehensive information based on your vast knowledge base and the current conversation. 
If you do not know the answer please acknowledge the fact by saying that you do not possess the information requested.If the conversation is summarised it will appear as System:. 
Current conversation:
Human: Hey there, Chatter
AI: What's up!
Human: Not much, just chillin before the busy day tomorrow
AI: Great to hear. Let me know if I can assist you in any way.
Human: As a mather of fact you can! How nice of you to ask.Here is what I have planned for tomorrow, please keep it in mind as I try to navigate the day.I will ask you questions once I go along. Plan: So my agenda for tomorrow is rather busy. First I need to engage into some fitness activies.
Once done, I will head to the bank around 11am. Sign some paperwork in order to obtain m

"Absolutely, Human! Tomorrow you have a busy schedule. Here's a summary:\n\n1. You plan to start your day with some fitness activities.\n2. Around 11 am, you will visit the bank to sign paperwork for obtaining a new credit card.\n3. Between 12 pm and 12:30 pm, you will be in the office checking on tasks and having lunch at 1 pm.\n4. In the afternoon, you expect a delivery from the bulb supplier.\n5. At 3 pm, you have a demo scheduled on RAG regarding an open-source Large Language Model (LLM).\n\nSo yes, you have fitness activities scheduled for tomorrow. If you need any further assistance or reminders throughout the day, feel free to ask!"

In [13]:
conversation.predict(input="Shooot! Thanks! I completely forgot about my demo at 15:00. Do you have any more info on that?")



> Entering new ConversationChain chain...
Prompt after formatting:
Human: <s>[INST]You are a helpful assistant ready to assist your human friend.Please provide comprehensive information based on your vast knowledge base and the current conversation. 
If you do not know the answer please acknowledge the fact by saying that you do not possess the information requested.If the conversation is summarised it will appear as System:. 
Current conversation:
System: 
The human greets the AI and shares their plans for the next day. They plan to engage in fitness activities first thing in the morning, visit the bank around 11am, attend an office meeting and have lunch at 1pm, receive a delivery from the bulb supplier in the afternoon, and give a demo on an open source language model at 3pm. The human requests that the AI remember these details.
AI: Thanks, got it, I will be happy to help!
Human: Hey AI, can you give me some summmary of my day tomorrow? Do I have fitness scheduled?
AI: Absolutely

"AI: Of course, Human! The demo you mentioned is for RAG (Rapid Analysis Group) and it's regarding an open-source Large Language Model (LLM). It's scheduled for 3 pm and you should prepare any necessary materials or presentations beforehand to ensure a successful demonstration. Let me know if there's anything specific you would like to review or discuss related to this demo. Additionally, I can set up a reminder for you closer to the event to make sure you don't forget."

In [14]:
conversation.predict(input="Hmmm.. RAG should be something to do with retreiving data from vector store and giving it as context to the language model..\
Are you sure it means Rapid Analysis Group?")



> Entering new ConversationChain chain...
Prompt after formatting:
Human: <s>[INST]You are a helpful assistant ready to assist your human friend.Please provide comprehensive information based on your vast knowledge base and the current conversation. 
If you do not know the answer please acknowledge the fact by saying that you do not possess the information requested.If the conversation is summarised it will appear as System:. 
Current conversation:
System: 
The human greets the AI and shares their plans for the next day, which include fitness activities, visiting the bank, attending an office meeting, receiving a delivery, and giving a demo on an open source language model. The AI confirms these plans and offers to provide reminders as needed.
Human: Shooot! Thanks! I completely forgot about my demo at 15:00. Do you have any more info on that?
AI: AI: Of course, Human! The demo you mentioned is for RAG (Rapid Analysis Group) and it's regarding an open-source Large Language Model (LLM

'AI: Yes, Human, you are correct in questioning the acronym "RAG." While I may have initially provided incorrect information due to insufficient context during our initial conversation, upon further investigation, I found out that "RAPID Analysis Group" could indeed be the name of the organization where you are scheduled to give the demo. This group focuses on using advanced analytical tools and techniques, including machine learning models like LLMs, to rapidly analyze large datasets and extract valuable insights. Therefore, it seems that the context of the demo does involve data retrieval and providing it as context to the language model. I apologize for any confusion earlier and am here to help clarify any doubts you might have. If you need any assistance in preparing for the demo or have any questions related to the topic, feel free to ask. Also, let me know if you would like me to set up a reminder for the demo.'

In [15]:
conversation.predict(input="Could it mean Retreive Augmented Generator?")



> Entering new ConversationChain chain...
Prompt after formatting:
Human: <s>[INST]You are a helpful assistant ready to assist your human friend.Please provide comprehensive information based on your vast knowledge base and the current conversation. 
If you do not know the answer please acknowledge the fact by saying that you do not possess the information requested.If the conversation is summarised it will appear as System:. 
Current conversation:
System: 

The human greets the AI and shares their plans for the next day, including fitness activities, visiting the bank, attending an office meeting, receiving a delivery, and giving a demo on an open source language model. The AI confirms these plans and offers to provide reminders as needed. When the human mentions they forgot about the details of their demo, the AI provides additional information and sets up a reminder.
Human: Hmmm.. RAG should be something to do with retreiving data from vector store and giving it as context to the 

'AI: That\'s another interesting possibility, Human! "Retrieve Augmented Generator (RAG)" is also a term used in some research communities, particularly in Natural Language Processing (NLP). In this context, RAG refers to a framework that assists in generating responses to text-based queries by first retrieving relevant information and then augmenting it to generate more accurate and informative answers. So, while it doesn\'t directly relate to the previous definition of RAPID Analysis Group, it\'s still an intriguing alternative interpretation. However, without knowing the specific context of your demo, it\'s difficult to say for certain which meaning applies. To clarify any doubts, you can always reach out to the organizers or check your email confirmation for more details. And, as always, I\'d be happy to help you prepare for the demo or answer any questions you might have regarding NLP or related topics. Additionally, if you\'d like me to set up a reminder for the demo, just let me

#### Summary:

After updating the template and the model works quite well with summarised memory. Good to know!

### Memory Types

Few types of memories are available. They have specific uses. Most of them are trying to conserve context by limiting the amount of memory carried forward in the conversation. Apart from the general `ConversationBufferMemory`, there are three different memories that conserve the amount of tokens carried forward:

- **ConversationBufferWindowMemory** - does this by supplying as parameter how many exchanges between the Human and AI to carry forward.
- **ConversationTokenBufferMemory** - same, but instead of number of conversation, number of tokens is specified. Note: here we also supply the llm so that the correct tokeniser is used to generate the tokens.
- **ConversationSummaryMemory** - This memory creates a summary of the conversation over time.

#### Other Types of Memory

Some other types that we will see in future:

- **Vector Data Memory** - Stores from conversation or other context in a vector DB and retreives the most relevant blocks of text - hmmm... RetreiverAugmentedGeneration (RAG) for context. Sounds familiar.
- **Entity Memories** - Using an LLM, it remembers details about specific entities.. weird??

You can also mix and match some types of memories together - Conversation + Entity to recall individuals. You can also store conversation in a normal SQL-like DB. But why would you do that if Vector DBs work quite well..



### The Chains!

The key building block of a LangChain application is naturally 'The Chain'!

- `LLMChain`
- Sequential Chains
  - `SimpleSequentialChain`
  - `SequentialChain`
- `RouterChain`

The chain usually combines LLM with a prompt and creates a building block that you can combine with other building blocks to carry on a sequence of operations on text or on other data.

Let's start experimenting.


In [51]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain
import pandas as pd
from langchain.memory import SimpleMemory

In [52]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [53]:
llm = ChatOpenAI(temperature=0.5, model="gpt-3.5-turbo-1106")
product_descr = "A helpfull assistant web platform that assists management of small companies\
in their team management and team motivation procedures"

In [54]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}? Provide just one example of name"
)
chain = LLMChain(llm=llm, prompt=prompt)
chain.run("Robots to assist in processing metal elements for the automotive industry")

/opt/conda/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'"MetalTech Robotics"'

Above LLMChain is the simplest possible chains, directly chaining prompt to llm and processing an input.
Another type is the `SimpleSequentialChain` which takes in only one other chain as input and returns an output. See below:

In [55]:
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the company with the following name: {name}"
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [57]:
simple_chain = SimpleSequentialChain(chains=[chain, chain_two], verbose=True)
simple_chain.run(product_descr)



> Entering new SimpleSequentialChain chain...
"TeamBoost Pro"
TeamBoost Pro provides innovative team-building solutions for companies, focusing on boosting productivity, collaboration, and overall team performance.

> Finished chain.


'TeamBoost Pro provides innovative team-building solutions for companies, focusing on boosting productivity, collaboration, and overall team performance.'

Now lets try something a little more fun. Let's take in a movie review, lets ask the LLM to rate it on a scale from 1-10, then summarise it, we will then supply the real user rating to the LLM and ask it to tell us how it did compared to the ground truth, presenting the summary and its response as json.

In [58]:
import json
import numpy as np
with open('./data/imdb_scraped_reviews_06.01-01.15.json','r') as f:
    movie_dict = json.loads(f.read())
movies = movie_dict["movies"]
rand_movie_index = np.random.randint(len(movies))
rand_review_index = np.random.randint(len(movies[rand_movie_index]["reviews"]))
movie = movies[rand_movie_index]
review = movie["reviews"][rand_review_index]
review_copy = review["copy"]
review_human_rating = review["stars"]

In [59]:
movie["title"]

'Wonka'

In [60]:
llm = ChatOpenAI(temperature=0.5, model="gpt-3.5-turbo-1106")

> Note: Trying the open LLM Mistral 7B. 

Load the model as `open_llm`:

In [61]:
from langchain import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

In [62]:
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", trust_remote_code=True, load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [71]:
generation_config = GenerationConfig.from_pretrained(model_name)
generation_config.max_new_tokens = 512
generation_config.temperature = 0.1
generation_config.top_p = 0.8
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15
generation_config.pad_token_id = tokenizer.eos_token_id

text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
)
open_llm = HuggingFacePipeline(pipeline=text_pipeline)

In [65]:
input_copy = "Take in a user review of a movie and rate the movie based on the sentiment of the review\
on a scale 1 to 10. 1 being a total flop, while 10 being a brilliant movie.\
Supply the rating and a one sentence explanation why you have rated it as such. Here is the review: {review}"
input_template = input_copy
input_template_open = f"<s>[INST]{input_copy}[INST]"

In [66]:
input_prompt = ChatPromptTemplate.from_template(input_template_open)
input_chain = LLMChain(llm=open_llm, prompt=input_prompt, output_key="review_rating")

In [67]:
grade_rating_copy = """Here is what your rating is of the movie based on the review: {review_rating} \
The human rating of the movie is actually: {human_rating} on the same scale 1 to 10. \
Can you grade how close your rating is based on the review's sentiment to the actual rating of the human reviewer {human_rating}\
on the A to F grading scale and provide the grade - \
'A' you have guessed the human rating completely, 'F' you have not guessed what the human rating is at all.
"""
grade_rating_template = grade_rating_copy
grade_rating_template_open = f"[INST]{grade_rating_copy}[/INST]"
grade_rating_prompt = ChatPromptTemplate.from_template(grade_rating_template_open)
grade_rating_chain = LLMChain(llm=open_llm, prompt=grade_rating_prompt, output_key="rating_grade")

In [68]:
final_response_copy = """Create a up to 100 words summary of the movie review.
Review:\n{review}
"""
final_response_template = final_response_copy
final_response_template_open = f"[INST]{final_response_copy}[/INST]"
final_response_prompt = ChatPromptTemplate.from_template(final_response_template_open)
final_response_chain = LLMChain(llm=open_llm, prompt=final_response_prompt, output_key="summary")

In [69]:
chain_of_chains = SequentialChain(
    chains=[input_chain, grade_rating_chain, final_response_chain],
    input_variables=["review"],
    output_variables=["review_rating", "rating_grade", "summary"],
    memory=SimpleMemory(memories={"title": movie["title"], "human_rating": review["stars"]}),
    verbose=True
)

In [73]:
r = chain_of_chains({"review": review["copy"]})



> Entering new SequentialChain chain...

> Finished chain.


In [75]:
r

{'review': 'I really had high hopes for this movie. Finally, I thought, Hollywood can take a step back from their incessant superhero schlock and do something genuinely creative and genuinely entertaining. And it seemed to be getting good reviews! Within the first thirty minutes, however, it becomes apparent this is a very drab, tedious affair. Somehow they managed to turn Willy Wonka into a joyless tale. If anything, it seemed to be slipping into "Les Miz" territory, and I was kind of hoping Olivia Colman might just go ahead and belt out "Master of the House." One of the few redeeming qualities is Hugh Grant as a sophisticated Oompa-Loompa. Aside from that, the supporting characters are horrible, the music instantly forgettable, and the plot is muddled and tedious, pulling at your emotions with tales of dead mothers and orphans rather than merry Wonkaesque mayhem. For Pete\'s sake, they were handed the keys to create Willy Wonka\'s origin story and they couldn\'t even make it watchabl

### Routing Chains - The Router Chain

When we need to make a decision what chain to load next based on some conditions we can use the Router Chain. To implement this, lets first create few different templates that we can load into the different chains.

In [76]:
llm = ChatOpenAI(temperature=0.01, model="gpt-3.5-turbo-1106")

Here we create different templates that the router chain will route to depending on what it understands the question to be relevant to.

In [77]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{input}"""

math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""

computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

We put this in a dictionary and from this dictionary we create the different chains with different templates

In [78]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "history", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [79]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

In [80]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt, verbose=True)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]

destination_str = "\n".join(destinations)

In [82]:
destination_str

'physics: Good for answering questions about physics\nmath: Good for answering math questions\nhistory: Good for answering history questions\ncomputer science: Good for answering computer science questions'

In [83]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

Here is the instruction for the router chain

In [84]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}
"""

In [85]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destination_str)
router_prompt = PromptTemplate(template=router_template, 
                               input_variables=["input"], 
                               output_parser=RouterOutputParser())

In [86]:
router_chain = LLMRouterChain.from_llm(llm=llm, prompt=router_prompt)
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains,
                         default_chain=default_chain,
                         verbose=True)


In [87]:
chain.run("Can you tell me when anti-matter was first coined as a term and by whom?")



> Entering new MultiPromptChain chain...


/opt/conda/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


history: {'input': 'Can you tell me when anti-matter was first coined as a term and by whom?'}

> Entering new LLMChain chain...
Prompt after formatting:
Human: You are a very good historian. You have an excellent knowledge of and understanding of people,events and contexts from a range of historical periods. You have the ability to think, reflect, debate, discuss and evaluate the past. You have a respect for historical evidenceand the ability to make use of it to support your explanations and judgements.

Here is a question:
Can you tell me when anti-matter was first coined as a term and by whom?

> Finished chain.

> Finished chain.


'The term "anti-matter" was first coined by the physicist Arthur Schuster in 1898. He used the term to describe the hypothetical counterpart to ordinary matter, which would have the opposite electrical charge. This concept was later developed and popularized by physicist Paul Dirac in the 1920s as part of his work on quantum mechanics.'

In [88]:
chain.run("what is anti-matter in simple terms?")



> Entering new MultiPromptChain chain...


/opt/conda/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


physics: {'input': 'what is anti-matter in simple terms?'}

> Entering new LLMChain chain...
Prompt after formatting:
Human: You are a very smart physics professor. You are great at answering questions about physics in a conciseand easy to understand manner. When you don't know the answer to a question you admit that you don't know.

Here is a question:
what is anti-matter in simple terms?

> Finished chain.

> Finished chain.


'Anti-matter is essentially the opposite of regular matter. It is made up of particles that have the same mass as regular matter particles, but with opposite charge. For example, the anti-particle of an electron (a regular matter particle) is called a positron, which has a positive charge instead of a negative charge. When matter and anti-matter come into contact, they annihilate each other, releasing a large amount of energy. Anti-matter is a fascinating area of study in physics and has potential applications in areas such as energy production and medical imaging.'

In [89]:
chain.run("what is GIL in python? Please explain in simple terms and also point out \
how is multi-threading achieved with its presence.")



> Entering new MultiPromptChain chain...


/opt/conda/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


computer science: {'input': 'What is the Global Interpreter Lock (GIL) in Python? Please explain in simple terms and also explain how multi-threading is achieved with its presence.'}

> Entering new LLMChain chain...
Prompt after formatting:
Human:  You are a successful computer scientist.You have a passion for creativity, collaboration,forward-thinking, confidence, strong problem-solving capabilities,understanding of theories and algorithms, and excellent communication skills. You are great at answering coding questions. You are so good because you know how to solve a problem by describing the solution in imperative steps that a machine can easily interpret and you know how to choose a solution that has a good balance between time complexity and space complexity. 

Here is a question:
What is the Global Interpreter Lock (GIL) in Python? Please explain in simple terms and also explain how multi-threading is achieved with its presence.

> Finished chain.

> Finished chain.


"The Global Interpreter Lock (GIL) in Python is a mechanism that allows only one thread to execute Python bytecode at a time. This means that even if you have multiple threads in your Python program, only one of them can execute at any given moment.\n\nThis might sound like a limitation, but it's actually a design choice that simplifies the implementation of the Python interpreter. The GIL ensures that memory management is handled safely, and it also makes it easier to work with C extensions and libraries.\n\nNow, when it comes to multi-threading in Python with the GIL present, you might think that it's not very effective. However, you can still achieve some level of parallelism by using multi-threading for I/O-bound tasks. This is because the GIL is released when a thread is waiting for I/O operations to complete, allowing other threads to execute.\n\nIn simple terms, the GIL in Python ensures that only one thread can execute Python code at a time, but you can still use multi-threadin

### LangChain - Q&A over Documents

An example might be a tool that would allow you to query a product catalog for items of interest. RAGs are great to complement LLMs with knowledge that was not part of their training, making them much more powerful and flexible.

Lets start by importing the necessary packages:

In [4]:
!pip install docarray

  Using cached rich-13.7.1-py3-none-any.whl.metadata (18 kB)
  Using cached types_requests-2.31.0.20240311-py3-none-any.whl.metadata (1.8 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached urllib3-2.2.1-py3-none-any.whl.metadata (6.4 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.2/270.2 kB 2.8 MB/s eta 0:00:0000:010:01
Using cached rich-13.7.1-py3-none-any.whl (240 kB)
Using cached types_requests-2.31.0.20240311-py3-none-any.whl (14 kB)
Using cached markdown_it_py-3.0.0-py3-none-any.whl (87 kB)
Using cached urllib3-2.2.1-py3-none-any.whl (121 kB)
Using cached mdurl-0.1.2-py3-none-any.whl (10.0 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.18
    Uninstalling urllib3-1.26.18:
      Successfully uninstalled urllib3-1.26.18
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is t

In [17]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [268]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader, JSONLoader
# In memory vector store as an alternative Chroma
from langchain.vectorstores import DocArrayInMemorySearch 
from IPython.display import display, Markdown
from langchain.llms import OpenAI
# Index creator for the vector store
from langchain.indexes import VectorstoreIndexCreator
from pydantic import ValidationError
import pandas as pd
import numpy as np
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

Lets load a CSV file for testing. We can use one of the files we created with Movie Sentiment

In [234]:
!pip install "langchain[docarray]"
!pip install jq

  Using cached docarray-0.32.1-py3-none-any.whl.metadata (30 kB)
  Using cached hnswlib-0.8.0.tar.gz (36 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Using cached docarray-0.32.1-py3-none-any.whl (215 kB)
  error: subprocess-exited-with-error
  
  × Building wheel for hnswlib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [60 lines of output]
      running bdist_wheel
      running build
      running build_ext
      creating tmp
      gcc -pthread -B /opt/conda/compiler_compat -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /opt/conda/include -fPIC -O2 -isystem /opt/conda/include -fPIC -I/opt/conda/include/python3.11 -c /tmp/tmp6ry2ao8k.cpp -o tmp/tmp6ry2ao8k.o -std=c++14
      gcc -pthread -B /opt/conda/compiler_compat -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /opt/conda/include -fPIC -O2 -isystem /opt/conda/include -fPIC -I/opt/conda/include/python3.11 -c /tmp/tmpzrb6j6

In [252]:
loader = CSVLoader(file_path="./data/movies_sentiment_06.01-01.15.csv")
json_loader = JSONLoader(file_path="./data/imdb_scraped_reviews_06.01-01.15.json", 
                         jq_schema='.movies[]', text_content=False)
# index = VectorstoreIndexCreator(vectorstore_cls=DocArrayInMemorySearch).from_loaders([loader])

In [70]:
# lets visualise the head of the SCV below using pandas.df
df = pd.read_csv("./data/movies_sentiment_06.01-01.15.csv")
# sort the df by total_sentiment column descending using a function to convert string to number
def convert_to_number(x):
    try:
        return float(x[:x.find('/')])
    except:
        return 0


# sort the df by total_sentiment column descending
df = df.sort_values(by="total_sentiment", ascending=False, key=lambda x: x.apply(convert_to_number))
df[:10]

,Unnamed: 0,title,stars,meta,votes,type,link,total_sentiment,total_stars
14,14,12th Fail,9.2,NaN,85925,Movie,https://www.imdb.com/title/tt23849204/,15 / 15,146 / 150
38,38,All of Us Strangers,8.1,89.0,4343,Movie,https://www.imdb.com/title/tt21192142/,15 / 15,123 / 150
4,4,Ted,8.1,NaN,9062,TV Mini Series,https://www.imdb.com/title/tt14824792/,15 / 15,141 / 150
5,5,Boy Swallows Universe,8.2,NaN,6728,TV Mini Series,https://www.imdb.com/title/tt10399902/,15 / 15,140 / 150
39,39,Godzilla Minus One,8.3,80.0,39321,Movie,https://www.imdb.com/title/tt23289160/,15 / 15,138 / 150
9,9,Society of the Snow,7.9,72.0,61274,Movie,https://www.imdb.com/title/tt16277242/,15 / 15,137 / 150
41,41,Merry Christmas,8.6,NaN,18323,Movie,https://www.imdb.com/title/tt15392282/,15 / 15,118 / 150
42,42,Mr Bates vs. The Post Office,8.8,NaN,6119,TV Mini Series,https://www.imdb.com/title/tt27867155/,14 / 15,143 / 150
47,47,American Fiction,7.9,82.0,5339,Movie,https://www.imdb.com/title/tt23561236/,14 / 15,110 / 150
20,20,The Iron Claw,8.1,73.0,14241,Movie,https://www.imdb.com/title/tt21064584/,14 / 15,125 / 150


In [42]:
query = "list all the movies with positive sentiment. consider the movies with positive sentiment to be \
any of the movies in the list with total_sentiment of more than 10. \
return a table in markdown with columns, name of the movies, \
sentiment score and link"
alt_query = "List the movies with stars numeric value of over 7 and return a table in markdown with columns, \
name of movie, stars, sentiment and link"

In [43]:
response = index.query(alt_query)

In [45]:
display(Markdown(response)) # for some reason something is not quite right. There are more movies with 10+ sentiment, stars also does not work


| Name of Movie | Stars | Sentiment | Link |
|---------------|-------|-----------|------|
| Animal | 6.8 | 5 / 15 | https://www.imdb.com/title/tt13751694/ |
| Maestro | 6.7 | 6 / 15 | https://www.imdb.com/title/tt5535276/ |

> Lets do it step by step as per the guide and double check where the issue is.

1. Lading - using the loader from above `CSVLoader`

In [303]:
data = json_loader.load()

2. Since the rows are very small, we will omit creating chunks
3. Lets use the OpenAI embeddings to create embeddings for the docs above

In [255]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()


In [142]:
embed = np.array(embeddings.embed_query("Hello world! My first name is Joseph."))
embed.ndim, embed.shape, embed[:5]

(1,
 (1536,),
 array([-0.01364027,  0.00315833, -0.00904409, -0.0185824 , -0.01099623]))

Now that we have the embeddings, we can create a new vector DB with the DocArrayInMemorySearch like so

4. Creating the DB with embeddings

In [305]:
def split_text(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100,
        length_function=len,
        add_start_index=True
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")
    return chunks
chunks = split_text(data)

Split 50 documents into 1310 chunks.


In [270]:
db = DocArrayInMemorySearch.from_documents(chunks, embeddings)

In [264]:
query = "Please suggest Mini Series with high ratings to watch."
r = db.similarity_search(query, k=7)
selected = list(r)

When we have the DB, we need to create a retreiver that will extract documents from it.

5. Creating a retreiver

In [314]:
retreiver = db.as_retriever(search_kwargs={"k": 7})

6. Now we create the LLM to use to generate the answer

In [307]:
llm = ChatOpenAI(temperature=0.1, model="gpt-3.5-turbo")

There are two ways to generate a response. Combining the documents supplied by the query into a long string and running through the llm:

7. Extract response
   - directly without a Retreiver Chain
  

In [164]:
qdocs = "\n\n".join([docs[i].page_content for i in range(len(docs))])

In [162]:
response = llm.call_as_llm(f"<docs>\n{qdocs}\n</docs>\nInstruction: Please list all the TV Series with over 7.0 stars \
from the between the <docs> and </docs> in a markdown \
table with the following columns: Title, Stars, Link, Sentiment.")
display(Markdown(response))


| Title                           | Stars | Link                                       | Sentiment |
|---------------------------------|-------|--------------------------------------------|-----------|
| Ted                             | 8.1   | [IMDb](https://www.imdb.com/title/tt14824792/) | 15/15     |
| The Brothers Sun                | 7.7   | [IMDb](https://www.imdb.com/title/tt17632862/) | 12/15     |
| Percy Jackson and the Olympians | 7.4   | [IMDb](https://www.imdb.com/title/tt12324366/) | 2/15      |
| The Curse                       | 7.1   | [IMDb](https://www.imdb.com/title/tt13623608/) | 4/15      |

In [163]:
print(f"<docs>\n{qdocs}\n</docs>\nInstruction: Please list all the TV Series with over 7.0 stars \
from the between the <docs> and </docs> in a markdown \
table with the following columns: Title, Stars, Link, Sentiment.")

<docs>
: 4
title: Ted
stars: 8.1
meta: N/A
votes: 9062
type: TV Mini Series
link: https://www.imdb.com/title/tt14824792/
total_sentiment: 15 / 15
total_stars: 141 / 150

: 7
title: Echo
stars: 6.1
meta: N/A
votes: 22711
type: TV Mini Series
link: https://www.imdb.com/title/tt13966962/
total_sentiment: 0 / 15
total_stars: 53 / 150

: 2
title: Fool Me Once
stars: 6.9
meta: N/A
votes: 32681
type: TV Mini Series
link: https://www.imdb.com/title/tt5611024/
total_sentiment: 3 / 15
total_stars: 60 / 150

: 13
title: The Brothers Sun
stars: 7.7
meta: N/A
votes: 11725
type: TV Series
link: https://www.imdb.com/title/tt17632862/
total_sentiment: 12 / 15
total_stars: 117 / 150

: 25
title: Berlin
stars: 7.1
meta: N/A
votes: 20578
type: TV Series
link: https://www.imdb.com/title/tt16288804/
total_sentiment: 1 / 15
total_stars: 58 / 150

: 10
title: Percy Jackson and the Olympians
stars: 7.4
meta: N/A
votes: 17609
type: TV Series
link: https://www.imdb.com/title/tt12324366/
total_sentiment: 2 / 15


7. Extra response
   - With a Retreival Chain

In [319]:
query_for_retreiver = "Based solely on the information provided as context, \
can you tell me what the reviews sentiment on Boy Swallows Universe is?"
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", 
                                 retriever=retreiver, 
                                 return_source_documents=True,
                                 verbose=True)
response = qa(query_for_retreiver)



> Entering new RetrievalQA chain...

> Finished chain.


In [312]:
response["source_documents"]

[Document(page_content='{"title": "Oppenheimer", "stars": 8.4, "link": "https://www.imdb.com/title/tt15398776/reviews", "meta": 89, "votes": 606834, "type": "Movie", "reviews": [{"copy": "One of the most anticipated films of the year for many people, myself included, Oppenheimer largely delivers. Much of it\'s great. I feel like I loved two of its three hours, and liked the other hour.... but it\'s that fact that stops me from adoring the entire thing. I know with Christopher Nolan\'s Dunkirk, that clicked on a second', metadata={'source': '/home/jovyan/work/data/imdb_scraped_reviews_06.01-01.15.json', 'seq_num': 12, 'start_index': 0}),
 Document(page_content="Strauss, with a few flashbacks to a couple of meetings he was involved in with Oppenheimer . It's a political drama, with not much drama. There isn't enough emotional connection to the characters to care about it. Will Oppenheimer lose his security clearance? Will Strauss get served by karma at his cabinet confirmation hearing? W

In [320]:
display(Markdown(response['result']))

Based on the reviews provided in the context, the sentiment on "Boy Swallows Universe" seems to be overwhelmingly positive. Reviewers have described it as a joy to watch, incredibly original and funny, and a celebration of Aussie talent. The positive comments about the characters, cast, and adaptation from the book to the screen indicate a favorable sentiment towards the TV Mini Series.

### Analysing and mesuring Langchain Apps Performance

### Outline:

- Example generation
- Manual evaluation (and debuging)
- LLM-assisted evaluation
- LangChain evaluation platform

In [17]:
!pip install jq
!pip install docarray
!pip install "langchain[docarray]"
!pip install tiktoken

  Using cached docarray-0.32.1-py3-none-any.whl.metadata (30 kB)
  Using cached hnswlib-0.8.0.tar.gz (36 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Using cached docarray-0.32.1-py3-none-any.whl (215 kB)
  error: subprocess-exited-with-error
  
  × Building wheel for hnswlib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [60 lines of output]
      running bdist_wheel
      running build
      running build_ext
      creating tmp
      gcc -pthread -B /opt/conda/compiler_compat -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /opt/conda/include -fPIC -O2 -isystem /opt/conda/include -fPIC -I/opt/conda/include/python3.11 -c /tmp/tmpi3re1xfm.cpp -o tmp/tmpi3re1xfm.o -std=c++14
      gcc -pthread -B /opt/conda/compiler_compat -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /opt/conda/include -fPIC -O2 -isystem /opt/conda/include -fPIC -I/opt/conda/include/python3.11 -c /tmp/tmpncqkvb

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

#### Create our QandA application

In [2]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.document_loaders import CSVLoader, JSONLoader
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
import json

Lets load the data from the previous example

In [25]:
json_loader = JSONLoader(file_path="./data/imdb_scraped_reviews_06.01-01.15.json", 
                         jq_schema='.movies[]', text_content=False)
data = json_loader.load()
def split_text(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=300,
        length_function=len,
        add_start_index=True
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")
    return chunks


### Simplify the documents a little bit

To test our evaluation, I have compiled a single document containing the movie reviews for Poor Things.

The reviews are taken from my data scraping the IMDB. Below is the data prep.

In [4]:
small_data = data[10:15]
dict = [json.loads(doc.page_content) for doc in small_data]
type(dict)
# dict_str = "\n".join([f"{doc['title']}\n\n{doc['copy']}\n\n===============\n" for doc in dict["reviews"]])
# custom_doc = [Document(page_content=dict_str, metadata={'source': 'custom_doc', 'row': 0})]
# custom_chunks = split_text(custom_doc)

list

In [26]:
list_of_docs = []
reviews = [{'reviews': doc['reviews'], 'title': doc['title']} for doc in dict ]
for doc_reviews in reviews:
    list_of_docs.append(
        Document(
            page_content=f"Movie: {doc_reviews['title']}\n\nReviews:\n\n" + "\n\n".join([f"{review['title']}\n\n{review['copy']}\n\n===============" for review in doc_reviews['reviews'][:5]]),
            metadata={'source': doc_reviews['title'], 'row': 0}
        ))
chunks = split_text(list_of_docs)
# doc_of_reviews = ["\n".join() for my_doc in dict for review in my_doc["reviews"]]
# doc_of_reviews
# doc_chunks = split_text(list_of_docs)


Split 5 documents into 63 chunks.


In [7]:
llm = ChatOpenAI(temperature=0.1, model="gpt-3.5-turbo")
embeddings = OpenAIEmbeddings()

/opt/conda/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/opt/conda/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [57]:
db = DocArrayInMemorySearch.from_documents(chunks, embeddings)

In [58]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", 
                                 retriever=db.as_retriever(), 
                                 verbose=True)


Ask the LLM a simple question to test if it works. It seems working. Lets proceed further.

In [144]:
qa.run('what is the best thing that people say about the Poor Things movie?')



> Entering new RetrievalQA chain...

> Finished chain.


'The best thing people say about the "Poor Things" movie is that it is a monumental achievement, the funniest, smartest, best-looking, and most entertaining film of 2023. It is described as a cinematic triumph that pushes boundaries, challenges expectations, and delivers an unforgettable experience. The film is praised for its storytelling brilliance, stellar performances, daring narrative, and radical inventiveness. It is considered a masterpiece and a potential cinematic milestone by future generations.'

1. The example generation

Below we prepare few examples and create QA pairs with ChatGPT. We assume this to be the ground truth. Although it in fact isn't. We can now use these generated QA pairs to compare with the answers of an open LLM model and ask the GPT to evaluate if the meaning match the generated ground truth.

In [8]:
from langchain.evaluation.qa import QAGenerateChain


In [9]:
example_gen_chain = QAGenerateChain.from_llm(llm)
# tt = [{"doc":t} for t in list_of_docs]

examples = example_gen_chain.apply_and_parse([{"doc":t} for t in list_of_docs])

/opt/conda/lib/python3.11/site-packages/langchain/chains/llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [12]:
examples

[{'qa_pairs': {'query': 'Based on the document provided, what are some common criticisms of the portrayal of Percy Jackson in the show?',
   'answer': 'Some common criticisms of the portrayal of Percy Jackson in the show include the fact that his character does not feel like the Percy from the books, as he lacks the snarky and quick-witted personality, his dialogue is not funny or witty, he chooses violence over using his words, and he does not stand up to Gabe like he does in the books. Additionally, viewers feel that the new Percy is not a compelling protagonist and they do not find themselves rooting for him.'}},
 {'qa_pairs': {'query': 'Who is credited as the lead actor to beat at this stage for early awards consideration in the film "Oppenheimer"?',
   'answer': 'Cillian Murphy is credited as the lead actor to beat at this stage for early awards consideration in the film "Oppenheimer".'}},
 {'qa_pairs': {'query': 'Who are the main characters in the movie "The Holdovers" and what a

In [12]:
# this can be used for debugging
# import langchain
# langchain.debug = False

2. The answers by an open LLM

In [13]:
from langchain import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
from langchain_community.embeddings import HuggingFaceEmbeddings
import torch
model_name = "../ext_models/Mistral-7B-Instruct-v0.2"

In [14]:
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", trust_remote_code=True, torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [15]:
embeddings = HuggingFaceEmbeddings(model_name="./data/embeddings/gte-large/", 
                                       model_kwargs={"device": "cuda"}, 
                                       encode_kwargs={"normalize_embeddings": True})

In [27]:
db = DocArrayInMemorySearch.from_documents(chunks, embeddings)

In [28]:
generation_config = GenerationConfig.from_pretrained(model_name)
generation_config.max_new_tokens = 512
generation_config.temperature = 0.1
generation_config.top_p = 0.9
generation_config.do_sample = True
generation_config.repetition_penalty = 1.1
generation_config.pad_token_id = tokenizer.eos_token_id
open_llm = HuggingFacePipeline(pipeline=pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
))

In [29]:
qa = RetrievalQA.from_chain_type(llm=open_llm, chain_type="stuff", 
                                 retriever=db.as_retriever(), 
                                 verbose=True)

In [71]:
qa.run("Who is the main teacher in the Holdovers?")

/opt/conda/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...

> Finished chain.


" Paul Hunham, played by Paul Giamatti, is the main teacher in 'The Holdovers'."

In [31]:
predictions = qa.apply([example["qa_pairs"] for example in examples])



> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


In [32]:
from langchain.evaluation.qa import QAEvalChain

In [33]:
eval_chain = QAEvalChain.from_llm(llm)
graded_outputs = eval_chain.evaluate([example["qa_pairs"] for example in examples], predictions)

In [34]:
graded_outputs

[{'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'INCORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'}]

In [35]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['results'])
    print()

Example 0:
Question: Based on the document provided, what are some common criticisms of the portrayal of Percy Jackson in the show?
Real Answer: Some common criticisms of the portrayal of Percy Jackson in the show include the fact that his character does not feel like the Percy from the books, as he lacks the snarky and quick-witted personality, his dialogue is not funny or witty, he chooses violence over using his words, and he does not stand up to Gabe like he does in the books. Additionally, viewers feel that the new Percy is not a compelling protagonist and they do not find themselves rooting for him.
Predicted Answer:  Some common criticisms of the portrayal of Percy Jackson in the show include his lack of humor and wit, his passive role in solving problems, and his tendency to resort to violence instead of using his words. Critics also note that he seems out of character in comparison to the Percy in the books. Additionally, some viewers have expressed disappointment that the act

### LangChain Agents

Agents are like gateways to access 3rd party data. Data stores, APIs and DBs, Search Engines, File Systems and so on. They facilitate the exploitation of LLMs as reasoning operatives that can help give advice or automate tasks. LLMs are not only data stores, but they can actually use this knowledge to provide somewhat intelligent output that can then be further incorporated in business logic. 

Lets try the following exciting cutting edge stuff:

- Using built in LangChain tools: DuckDuckGo search and Wikipedia
- Defining your own tools

First some setting up. **Important:** we set the temperature to 0, since we are using the openAI LLM as a reasoning agent. We want the reasoning to be as good and precise as possible removing all the randomness.



In [11]:
!pip install langchain_experimental
!pip install numexpr
!pip install -U wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=e515f7eb127b4aaf0c7da0a19b8f2e1f4bd824a1e68efd6edd09ab4436523b84
  Stored in directory: /home/jovyan/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


In [31]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI

In [32]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [33]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
tools = load_tools(["llm-math", "wikipedia"], llm)

In [13]:
agent = initialize_agent(
            tools, 
            llm, 
            agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
            handle_parsing_errors=True,
            verbose = True)

/opt/conda/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


#### Math problems with the llm-math

In [14]:
agent("what is 25% of 300?")

/opt/conda/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: To find 25% of 300, we can multiply 300 by 0.25.

Action:
```
{
  "action": "Calculator",
  "action_input": "300 * 0.25"
}
```


Observation: Answer: 75.0
Thought:I now know the final answer

Final Answer: 25% of 300 is 75.

> Finished chain.


{'input': 'what is 25% of 300?', 'output': '25% of 300 is 75.'}

In [15]:
agent("if I know that one hour of a Amazon AWS machine costs 10c, \
what will my bill be for one month in dollars if the machine is \
running the whole time?")



> Entering new AgentExecutor chain...
Thought: To calculate the total cost for one month, I need to find out how many hours are in a month and then multiply that by the cost per hour.

Action:
```
{
  "action": "Calculator",
  "action_input": "24 * 30 * 0.10"
}
```


Observation: Answer: 72.0
Thought:Final Answer: The bill for one month in dollars if the machine is running the whole time will be $72.

> Finished chain.


{'input': 'if I know that one hour of a Amazon AWS machine costs 10c, what will my bill be for one month in dollars if the machine is running the whole time?',
 'output': 'The bill for one month in dollars if the machine is running the whole time will be $72.'}

#### Wikipedia search

Note to self: as of today the 10th of April, GPT 3.5 still thinks the monarch is Queen Elizabeth II. Below is the correct answer as of today.

In [17]:
agent("Who is the King of England?")



> Entering new AgentExecutor chain...
Thought: I will use Wikipedia to find information about the current King of England.
Action:
```
{
  "action": "wikipedia",
  "action_input": "King of England"
}
```
Observation: Page: Monarchy of the United Kingdom
Summary: The monarchy of the United Kingdom, commonly referred to as the British monarchy, is the constitutional form of government by which a hereditary sovereign reigns as the head of state of the United Kingdom, the Crown Dependencies and the British Overseas Territories. The current monarch is King Charles III, who ascended the throne on 8 September 2022, upon the death of his mother, Queen Elizabeth II.
The monarch and their immediate family undertake various official, ceremonial, diplomatic and representational duties. Although formally the monarch has authority over the government—which is known as "His/Her Majesty's Government"—this power may only be used according to laws enacted in Parliament and within constraints of conven

{'input': 'Who is the King of England?', 'output': 'King Charles III'}

#### Writing Python


In [18]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [26]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""")



> Entering new AgentExecutor chain...
We can use the `sorted()` function in Python to sort the list of customers based on their last name and then first name.
Action: Python_REPL
Action Input: sorted([['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']], key=lambda x: (x[1], x[0]))
Observation: 
Thought:I now know the final answer
Final Answer: [['Jen', 'Ayai'], ['Harrison', 'Chase'], ['Lang', 'Chain'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Dolly', 'Too']]

> Finished chain.


"[['Jen', 'Ayai'], ['Harrison', 'Chase'], ['Lang', 'Chain'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Dolly', 'Too']]"

In [24]:
sorted_list = sorted(customer_list, key=lambda x: (x[1]))

In [25]:
sorted_list

[['Jen', 'Ayai'],
 ['Lang', 'Chain'],
 ['Harrison', 'Chase'],
 ['Elle', 'Elem'],
 ['Trance', 'Former'],
 ['Geoff', 'Fusion'],
 ['Dolly', 'Too']]

In [27]:
agent("tell me what the factorial of 7 is")



> Entering new AgentExecutor chain...
I can calculate the factorial of 7 by multiplying all numbers from 1 to 7.
Action: Python_REPL
Action Input: import math
Observation: 
Thought:I imported the math module to use the factorial function.
Action: Python_REPL
Action Input: math.factorial(7)
Observation: 
Thought:The factorial of 7 is 5040.
Final Answer: 5040

> Finished chain.


{'input': 'tell me what the factorial of 7 is', 'output': '5040'}

In [37]:
# this can be used for debugging
# import langchain
# langchain.debug = False

#### Creating a custom tool with `tool` decorator

In [40]:
from langchain.agents import tool
from datetime import date

@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathematics should occur \
    outside this function."""
    return str(date.today())

In [41]:
agent = initialize_agent(tools + [time], 
                         llm, 
                         agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, 
                         verbose = True, 
                         handle_parsing_errors=True)
agent("What is the date today?")



> Entering new AgentExecutor chain...
Thought: I can use the time tool to find out today's date.
Action:
```
{
  "action": "time",
  "action_input": ""
}
``` 

Observation: 2024-04-10
Thought:Could not parse LLM output: Final Answer
Observation: Invalid or incomplete response
Thought:I will correct the previous action and try again to get today's date.
Action:
```
{
  "action": "time",
  "action_input": ""
}
``` 

Observation: 2024-04-10
Thought:Final Answer: Today's date is April 10, 2024.

> Finished chain.


{'input': 'What is the date today?',
 'output': "Today's date is April 10, 2024."}